In [1]:
#!pip install git+https://github.com/deepmipt/bert.git@feat/keras tf-nightly-gpu deeppavlov

In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'1.14.1-dev20190519'

In [3]:
tf.compat.v1.enable_eager_execution()

In [4]:
tf.compat.v1.executing_eagerly()

True

# Model construction

In [5]:
from bert_dp import bert, tokenization

In [6]:
bert_model = bert.BERT(name='bert')

W0520 12:04:36.287844 139842443900736 deprecation_wrapper.py:119] From /home/nab/PycharmProjects/work/bert/bert_dp/embeddings.py:27: The name tf.keras.initializers.TruncatedNormal is deprecated. Please use tf.compat.v1.keras.initializers.TruncatedNormal instead.



In [7]:
BATCH_SIZE = 64

In [8]:
# some data to pass through model in order to construct it
toy_inputs = tf.constant([[1, 3, 2, 0], [1, 2, 3, 1]] * (BATCH_SIZE // 2))
# toy_mask = tf.cast(tf.not_equal(toy_inputs, 0), dtype=tf.int32)

In [9]:
bert_model(inputs=toy_inputs,
#            mask=toy_mask
          )

<tf.Tensor: id=4455, shape=(64, 768), dtype=float32, numpy=
array([[-0.88269734, -0.09706366,  3.3584752 , ...,  0.2638248 ,
        -0.28027695,  1.0510167 ],
       [-1.1150815 , -0.02016854,  2.8737    , ...,  0.33661264,
        -0.68565625,  0.67546195],
       [-0.88269734, -0.09706366,  3.3584752 , ...,  0.2638248 ,
        -0.28027695,  1.0510167 ],
       ...,
       [-1.1150815 , -0.02016854,  2.8737    , ...,  0.33661264,
        -0.68565625,  0.67546195],
       [-0.88269734, -0.09706366,  3.3584752 , ...,  0.2638248 ,
        -0.28027695,  1.0510167 ],
       [-1.1150815 , -0.02016854,  2.8737    , ...,  0.33661264,
        -0.68565625,  0.67546195]], dtype=float32)>

In [10]:
bert_model.summary()

Model: "bert"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embeddings (BERTCombinedEmbe multiple                  92206848  
_________________________________________________________________
embeddings/dropout (Dropout) multiple                  0         
_________________________________________________________________
embeddings/LayerNorm (LayerN multiple                  1536      
_________________________________________________________________
encoder (Sequential)         multiple                  85054464  
_________________________________________________________________
pooler/dense (Dense)         multiple                  590592    
Total params: 177,853,440
Trainable params: 177,853,440
Non-trainable params: 0
_________________________________________________________________


# Checkpoint loading

In [11]:
tf.keras.utils.get_file(
    fname='rubert_cased_L-12_H-768_A-12_v1.tar.gz',
    origin='http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_v1.tar.gz',
    cache_subdir='models',
    extract=True,
    cache_dir='.'
)

'./models/rubert_cased_L-12_H-768_A-12_v1.tar.gz'

In [12]:
init_checkpoint = 'models/rubert_cased_L-12_H-768_A-12_v1/bert_model.ckpt'
var_names = tf.train.list_variables(init_checkpoint)
check_point = tf.train.load_checkpoint(init_checkpoint)

In [13]:
checkpoint_vars = set()
for v, _ in var_names:
    checkpoint_vars.add(v)

In [14]:
checkpoint_vars

{'bert/embeddings/LayerNorm/beta',
 'bert/embeddings/LayerNorm/gamma',
 'bert/embeddings/position_embeddings',
 'bert/embeddings/position_embeddings/AdamWeightDecayOptimizer',
 'bert/embeddings/position_embeddings/AdamWeightDecayOptimizer_1',
 'bert/embeddings/token_type_embeddings',
 'bert/embeddings/word_embeddings',
 'bert/encoder/layer_0/attention/output/LayerNorm/beta',
 'bert/encoder/layer_0/attention/output/LayerNorm/gamma',
 'bert/encoder/layer_0/attention/output/dense/bias',
 'bert/encoder/layer_0/attention/output/dense/kernel',
 'bert/encoder/layer_0/attention/self/key/bias',
 'bert/encoder/layer_0/attention/self/key/kernel',
 'bert/encoder/layer_0/attention/self/query/bias',
 'bert/encoder/layer_0/attention/self/query/kernel',
 'bert/encoder/layer_0/attention/self/value/bias',
 'bert/encoder/layer_0/attention/self/value/kernel',
 'bert/encoder/layer_0/intermediate/dense/bias',
 'bert/encoder/layer_0/intermediate/dense/kernel',
 'bert/encoder/layer_0/output/LayerNorm/beta',
 

In [15]:
len(checkpoint_vars)

209

In [16]:
model_vars = set()
for v in bert_model.variables:
    model_vars.add(v.name.split(':')[0])

In [17]:
model_vars

{'bert/embeddings/LayerNorm/beta',
 'bert/embeddings/LayerNorm/gamma',
 'bert/embeddings/position_embeddings',
 'bert/embeddings/token_type_embeddings',
 'bert/embeddings/word_embeddings',
 'bert/encoder/layer_0/attention/output/LayerNorm/beta',
 'bert/encoder/layer_0/attention/output/LayerNorm/gamma',
 'bert/encoder/layer_0/attention/output/dense/bias',
 'bert/encoder/layer_0/attention/output/dense/kernel',
 'bert/encoder/layer_0/attention/self/key/bias',
 'bert/encoder/layer_0/attention/self/key/kernel',
 'bert/encoder/layer_0/attention/self/query/bias',
 'bert/encoder/layer_0/attention/self/query/kernel',
 'bert/encoder/layer_0/attention/self/value/bias',
 'bert/encoder/layer_0/attention/self/value/kernel',
 'bert/encoder/layer_0/intermediate/dense/bias',
 'bert/encoder/layer_0/intermediate/dense/kernel',
 'bert/encoder/layer_0/output/LayerNorm/beta',
 'bert/encoder/layer_0/output/LayerNorm/gamma',
 'bert/encoder/layer_0/output/dense/bias',
 'bert/encoder/layer_0/output/dense/kernel

In [18]:
len(model_vars)

199

In [19]:
bert_model.load_weights(init_checkpoint)

W0520 12:05:21.209319 139842443900736 deprecation.py:323] From /home/nab/PycharmProjects/work/venv/lib/python3.6/site-packages/tensorflow/python/training/tracking/util.py:1200: NameBasedSaverStatus.__init__ (from tensorflow.python.training.tracking.util) is deprecated and will be removed in a future version.
Instructions for updating:
Restoring a name-based tf.train.Saver checkpoint using the object-based restore API. This mode uses global names to match variables, and so is somewhat fragile. It also adds new restore ops to the graph each time it is called when graph building. Prefer re-encoding training checkpoints in the object-based format: run save() on the object-based saver (the same one this message is coming from) and use that checkpoint in the future.


# Paraphrase data

In [20]:
tf.keras.utils.get_file(
    fname='paraphraser.zip',
    origin='http://files.deeppavlov.ai/datasets/paraphraser.zip',
    cache_subdir='dataset',
    extract=True,
    cache_dir='.'
)

'./dataset/paraphraser.zip'

In [21]:
tf.keras.utils.get_file(
    fname='paraphraser_gold.zip',
    origin='http://files.deeppavlov.ai/datasets/paraphraser_gold.zip',
    cache_subdir='dataset',
    extract=True,
    archive_format='zip',
    cache_dir='.'
)

'./dataset/paraphraser_gold.zip'

In [22]:
def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

In [23]:
def data_gen(data_type='train'):
    from deeppavlov.dataset_readers.paraphraser_reader import ParaphraserReader
    from bert_dp.tokenization import FullTokenizer
    ds = ParaphraserReader().read(data_path='dataset', do_lower_case=False)
    tokenizer = FullTokenizer(vocab_file='models/rubert_cased_L-12_H-768_A-12_v1/vocab.txt', do_lower_case=False)
    for pair, label in ds[data_type]:
        t1 = tokenizer.tokenize(pair[0])
        t2 = tokenizer.tokenize(pair[1])
        _truncate_seq_pair(t1, t2, 512)
        s = ['[CLS]'] + t1 + ['[SEP]'] + t2 + ['[SEP]']
        yield tokenizer.convert_tokens_to_ids(s), label

In [24]:
train_ds = tf.data.Dataset.from_generator(data_gen,
                                          output_types=(tf.int32, tf.int32),
                                          output_shapes=(tf.TensorShape([None]), tf.TensorShape([])))


W0520 12:05:29.711235 139842443900736 deprecation.py:323] From /home/nab/PycharmProjects/work/venv/lib/python3.6/site-packages/tensorflow/python/data/ops/dataset_ops.py:494: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


In [25]:
batched_train_ds = train_ds.padded_batch(batch_size=BATCH_SIZE,
                                         padded_shapes=(tf.constant([-1], dtype=tf.int64), ()),
                                         drop_remainder=True)

In [26]:
test_ds = tf.data.Dataset.from_generator(data_gen,
                                         output_types=(tf.int32, tf.int32),
                                         output_shapes=(tf.TensorShape([None]), tf.TensorShape([])),
                                         args=('test',))

In [27]:
batched_test_ds = train_ds.padded_batch(batch_size=BATCH_SIZE,
                                        padded_shapes=(tf.constant([-1], dtype=tf.int64), ()),
                                        drop_remainder=True)

# Training

In [28]:
paraphrase_detector = tf.keras.Sequential([bert_model, tf.keras.layers.Dense(1, activation='sigmoid')])

In [29]:
from bert_dp.weight_decay_optimizers import AdamW

In [30]:
learning_rate= tf.keras.optimizers.schedules.PolynomialDecay(initial_learning_rate=2e-05,
                                                             # n. of batches per epoch * ~2 epochs for convergence 
                                                             decay_steps=208,
                                                             end_learning_rate=0.0)

In [31]:
paraphrase_detector.compile(optimizer=AdamW(weight_decay=0.01,
                                            learning_rate=learning_rate),
                            loss='binary_crossentropy',
                            metrics=[tf.keras.metrics.Accuracy(),
                                     tf.keras.metrics.Precision(),
                                     tf.keras.metrics.Recall()])

In [39]:
for x_batch, y_batch in batched_train_ds:
    paraphrase_detector.train_on_batch(x=x_batch, y=y_batch)
#                         epochs=3,
#                         callbacks=[tf.keras.callbacks.EarlyStopping()],
#                         validation_split=0.07,  # ~500 samples
#                         steps_per_epoch=100
#                                       )

InvalidArgumentError: Incompatible shapes: [64,1] vs. [2,1]
	 [[{{node training/AdamW/gradients/loss/output_1_loss/mul_grad/BroadcastGradientArgs}}]] [Op:__inference_keras_scratch_graph_38952]

In [32]:
paraphrase_detector.fit(batched_train_ds,
                        epochs=3,
                        callbacks=[tf.keras.callbacks.EarlyStopping()],
#                         validation_split=0.07,  # ~500 samples
                        steps_per_epoch=100)

Epoch 1/3


W0520 12:06:01.740266 139838590265088 deprecation_wrapper.py:119] From /home/nab/PycharmProjects/work/bert/bert_dp/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W0520 12:06:05.062074 139842443900736 deprecation.py:323] From /home/nab/PycharmProjects/work/venv/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0520 12:06:09.825000 139842443900736 deprecation.py:323] From /home/nab/PycharmProjects/work/venv/lib/python3.6/site-packages/tensorflow/python/keras/optimizer_v2/learning_rate_schedule.py:409: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


  1/100 [..............................] - ETA: 43:59 - loss: 0.7322 - accuracy: 0.0000e+00 - precision: 0.8333 - recall: 0.3636

InvalidArgumentError: Incompatible shapes: [64,12,38,38] vs. [64,1,1,31]
	 [[{{node training/AdamW/gradients/sequential/bert/encoder/layer_0/attention/add_grad/BroadcastGradientArgs}}]] [Op:__inference_keras_scratch_graph_39198]